<a href="https://colab.research.google.com/github/N1sChaLLL/Prediction-of-Hallucinations-in-Large-Learning-Models/blob/main/BERT_EVALUATION_METRICS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load training data
train_data = pd.read_csv('train.csv')  # replace 'train.csv' with your actual file path
prompts_train = train_data['Prompt'].tolist()
answers_train = train_data['Answer'].tolist()
labels_train = train_data['Target'].tolist()  # Target is present in train.csv

# Load test data (this does not have 'Target')
test_data = pd.read_csv('test.csv')  # replace 'test.csv' with your actual file path
prompts_test = test_data['Prompt'].tolist()
answers_test = test_data['Answer'].tolist()

# Check the first few rows of both datasets
print("Training Data:")
print(train_data.head())
print("\nTest Data:")
print(test_data.head())


Training Data:
      Id                                             Prompt  \
0  11527  [INST] You are an AI assistant that helps peop...   
1   7322  [INST] You are an AI assistant. You will be gi...   
2  11742  [INST] You are an AI assistant. You will be gi...   
3  20928  [INST] You are an AI assistant. User will you ...   
4  25830  [INST] You are an AI assistant. User will you ...   

                                              Answer  Target  
0  Step-by-step reasoning process:\n1. Randy spen...       0  
1  What is the temperature at which hypothermia b...       0  
2  Answer: c) No. \n\nThe hypothesis is false bec...       0  
3                                         Prismatoid       0  
4                                             Case B       0  

Test Data:
      Id                                             Prompt  \
0  20568  [INST] You are an AI assistant. You will be gi...   
1  17686  question:Question: This article: According to ...   
2  13035  [INST] You are an

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Set the model to evaluation mode (disable gradient calculation)
model.eval()


c:\ProgramData\anaconda3\envs\saturn_2\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
import numpy as np

# Load data (train.csv)
data = pd.read_csv('train.csv')

# Extract prompts and answers
prompts_train = data['Prompt'].tolist()
answers_train = data['Answer'].tolist()

# Check for rows where either the prompt or answer is missing or empty
filtered_prompts, filtered_answers = [], []
for prompt, answer in zip(prompts_train, answers_train):
    if isinstance(prompt, str) and isinstance(answer, str) and prompt.strip() and answer.strip():
        filtered_prompts.append(prompt)
        filtered_answers.append(answer)

# Load the BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to get BERT embeddings with batch processing
def get_bert_embeddings(text_list, batch_size=32):
    embeddings = []
    for i in range(0, len(text_list), batch_size):
        batch_text = text_list[i:i+batch_size]

        # Tokenize the batch and convert to PyTorch tensors
        inputs = tokenizer(batch_text, return_tensors='pt', padding=True, truncation=True, max_length=128)

        # Forward pass through BERT to get embeddings
        with torch.no_grad():
            outputs = model(**inputs)
            # Take the mean of the last hidden states to get the sentence embedding
            batch_embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
            embeddings.append(batch_embeddings)

    # Concatenate all batches to form the final embedding matrix
    return np.concatenate(embeddings)

# Generate embeddings for filtered training data
print("Generating BERT embeddings for prompts...")
prompt_embeddings_train = get_bert_embeddings(filtered_prompts)

print("Generating BERT embeddings for answers...")
answer_embeddings_train = get_bert_embeddings(filtered_answers)

# Check the shape of the embeddings
print("Filtered prompt embeddings shape:", prompt_embeddings_train.shape)
print("Filtered answer embeddings shape:", answer_embeddings_train.shape)

# Concatenate prompt and answer embeddings for training data
combined_embeddings_train = np.concatenate([prompt_embeddings_train, answer_embeddings_train], axis=1)

# Check final embeddings shape
print("Combined embeddings shape:", combined_embeddings_train.shape)

# Save the embeddings for later use if needed
np.save('combined_embeddings_train.npy', combined_embeddings_train)


c:\ProgramData\anaconda3\envs\saturn_2\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Generating BERT embeddings for prompts...
Generating BERT embeddings for answers...
Filtered prompt embeddings shape: (16668, 768)
Filtered answer embeddings shape: (16668, 768)
Combined embeddings shape: (16668, 1536)


In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

# Get labels (Targets) from your dataset
labels_train = data['Target'].tolist()

# Filter labels to match filtered prompts and answers
filtered_labels = []
for i, (prompt, answer) in enumerate(zip(prompts_train, answers_train)):
    if isinstance(prompt, str) and isinstance(answer, str) and prompt.strip() and answer.strip():
        filtered_labels.append(labels_train[i])

# Convert the filtered labels to a NumPy array
labels_train_np = np.array(filtered_labels)
print("Filtered Labels shape:", labels_train_np.shape)

# Split the combined embeddings and labels into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(combined_embeddings_train, labels_train_np, test_size=0.2, random_state=42)

# Print shapes to confirm
print("Training data shape:", X_train.shape)
print("Validation data shape:", X_val.shape)
print("Training labels shape:", y_train.shape)
print("Validation labels shape:", y_val.shape)



Filtered Labels shape: (16668,)
Training data shape: (13334, 1536)
Validation data shape: (3334, 1536)
Training labels shape: (13334,)
Validation labels shape: (3334,)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Initialize the Logistic Regression model
clf = LogisticRegression(max_iter=1000)  # Increase max_iter to ensure convergence

# Train the classifier on the training set
print("Training the Logistic Regression classifier...")
clf.fit(X_train, y_train)

# Predict on the validation set
y_pred = clf.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

# Get a detailed classification report (precision, recall, F1-score)
print("Classification Report:")
print(classification_report(y_val, y_pred, target_names=['No Hallucination', 'Hallucination']))


Training the Logistic Regression classifier...
Validation Accuracy: 94.48%
Classification Report:
                  precision    recall  f1-score   support

No Hallucination       0.96      0.98      0.97      3176
   Hallucination       0.33      0.16      0.21       158

        accuracy                           0.94      3334
       macro avg       0.64      0.57      0.59      3334
    weighted avg       0.93      0.94      0.94      3334



In [ ]:
import joblib

# Save the trained Logistic Regression model to a file
model_filename = 'logistic_regression_model.pkl'
joblib.dump(clf, model_filename)

print(f"Model saved to {model_filename}")


Model saved to logistic_regression_model.pkl


In [ ]:
# Ensure the test data is properly cleaned by filtering out invalid rows
filtered_prompts_test, filtered_answers_test = [], []
for prompt, answer in zip(prompts_test, answers_test):
    if isinstance(prompt, str) and isinstance(answer, str) and prompt.strip() and answer.strip():
        filtered_prompts_test.append(prompt)
        filtered_answers_test.append(answer)

# Generate BERT embeddings for the filtered test data (prompts and answers)
print("Generating BERT embeddings for filtered test prompts...")
prompt_embeddings_test = get_bert_embeddings(filtered_prompts_test)

print("Generating BERT embeddings for filtered test answers...")
answer_embeddings_test = get_bert_embeddings(filtered_answers_test)

# Concatenate prompt and answer embeddings for the test set
combined_embeddings_test = np.concatenate([prompt_embeddings_test, answer_embeddings_test], axis=1)

# Load the trained model
clf = joblib.load('logistic_regression_model.pkl')

# Make predictions on the test data
test_predictions = clf.predict(combined_embeddings_test)

# Output the predictions (1: Hallucination, 0: No Hallucination)
print("Test Predictions:", test_predictions)

# Save predictions to a CSV file if needed
output_df = pd.DataFrame({'Id': test_data['Id'][:len(test_predictions)], 'Prediction': test_predictions})
output_df.to_csv('test_predictions.csv', index=False)

print("Predictions saved to test_predictions.csv")


Generating BERT embeddings for filtered test prompts...
Generating BERT embeddings for filtered test answers...
Test Predictions: [0 1 0 ... 0 0 0]
Predictions saved to test_predictions.csv


In [ ]:
# Assuming data['Target'] contains the target labels
y_train = data['Target'].tolist()
from sklearn.preprocessing import LabelEncoder


# Filter the prompts, answers, and target labels simultaneously
filtered_prompts, filtered_answers, filtered_labels = [], [], []
for prompt, answer, label in zip(prompts_train, answers_train, y_train):
    if isinstance(prompt, str) and isinstance(answer, str) and prompt.strip() and answer.strip():
        filtered_prompts.append(prompt)
        filtered_answers.append(answer)
        filtered_labels.append(label)

# Convert the labels to numeric encoding
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(filtered_labels)

# Now that the filtering is consistent, you can split the data
X = np.concatenate([prompt_embeddings_train, answer_embeddings_train], axis=1)  # Make sure to use filtered embeddings

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# Proceed with evaluating classifiers as before


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import ExtraTreesClassifier, BaggingClassifier
from sklearn.linear_model import RidgeClassifier, Perceptron

# List of classifiers
classifiers = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Naive Bayes": GaussianNB(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "MLP Classifier": MLPClassifier(max_iter=1000),
    "LDA": LinearDiscriminantAnalysis(),
    "QDA": QuadraticDiscriminantAnalysis(),
    "Extra Trees": ExtraTreesClassifier(),
    "Bagging Classifier": BaggingClassifier(),
    "Ridge Classifier": RidgeClassifier(),
    "Perceptron": Perceptron(),
}

# Initialize a DataFrame to store the results
results_df = pd.DataFrame(columns=[
    "Classifier", "Train Accuracy", "Val Accuracy", "Train F1", "Val F1",
    "Train Precision", "Val Precision", "Train Recall", "Val Recall"
])

# Function to evaluate and store metrics
def evaluate_classifier(clf_name, clf, X_train, y_train, X_val, y_val):
    # Train the classifier
    clf.fit(X_train, y_train)

    # Predictions
    y_train_pred = clf.predict(X_train)
    y_val_pred = clf.predict(X_val)

    # Calculate metrics
    train_acc = accuracy_score(y_train, y_train_pred)
    val_acc = accuracy_score(y_val, y_val_pred)
    train_f1 = f1_score(y_train, y_train_pred, average='weighted')
    val_f1 = f1_score(y_val, y_val_pred, average='weighted')
    train_precision = precision_score(y_train, y_train_pred, average='weighted')
    val_precision = precision_score(y_val, y_val_pred, average='weighted')
    train_recall = recall_score(y_train, y_train_pred, average='weighted')
    val_recall = recall_score(y_val, y_val_pred, average='weighted')

    # Append the results to the DataFrame
    results_df.loc[len(results_df)] = [
        clf_name, train_acc, val_acc, train_f1, val_f1,
        train_precision, val_precision, train_recall, val_recall
    ]

# Loop through classifiers and evaluate each one
for clf_name, clf in classifiers.items():
    print(f"Evaluating {clf_name}...")
    evaluate_classifier(clf_name, clf, X_train, y_train, X_val, y_val)

# Display the results
print(results_df)

# Optionally, save the results to a CSV file
results_df.to_csv('classifier_comparison_results.csv', index=False)


Evaluating Logistic Regression...
Evaluating Random Forest...
Evaluating SVM...
Evaluating KNN...
Evaluating Decision Tree...
Evaluating Naive Bayes...
Evaluating Gradient Boosting...
Evaluating AdaBoost...


c:\ProgramData\anaconda3\envs\saturn_2\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Evaluating MLP Classifier...
Evaluating LDA...
Evaluating QDA...


c:\ProgramData\anaconda3\envs\saturn_2\lib\site-packages\sklearn\discriminant_analysis.py:947: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\ProgramData\anaconda3\envs\saturn_2\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\ProgramData\anaconda3\envs\saturn_2\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Evaluating Extra Trees...
Evaluating Bagging Classifier...
Evaluating Ridge Classifier...
Evaluating Perceptron...
             Classifier  Train Accuracy  Val Accuracy  Train F1    Val F1  \
0   Logistic Regression        0.961902      0.941812  0.953934  0.932226   
1         Random Forest        0.999400      0.950510  0.999401  0.932070   
2                   SVM        0.949528      0.949010  0.927834  0.926910   
3                   KNN        0.954627      0.946911  0.942517  0.933160   
4         Decision Tree        0.999475      0.900420  0.999476  0.907193   
5           Naive Bayes        0.732563      0.720156  0.805632  0.797023   
6     Gradient Boosting        0.958077      0.948710  0.945188  0.931732   
7              AdaBoost        0.948928      0.943311  0.934153  0.928997   
8        MLP Classifier        0.998500      0.934913  0.998507  0.928916   
9                   LDA        0.960027      0.932513  0.957195  0.930067   
10                  QDA        0.94705